Read in the cleaned traffic data for each of the store and adding prediction dates
--

In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from xgboost import plot_importance
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split
import math, os
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
store_list = [7, 8, 9, 10, 11, 14, 17, 18, 19, 25, 31, 32, 33, 34, 35, 37, 39, 41, 43, 48, 49, 52, 54, 55, 57, 59, 60, 62, 63, 64, 65, 66, 67, 68, 70, 71, 72, 73, 75, 77, 79, 82, 83, 84, 86, 89, 91, 95, 96, 97, 99, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 117, 118, 119, 121, 123, 125, 126, 127, 129, 130, 132, 134, 136, 138, 139, 140, 141, 142, 143, 147, 148, 149, 153, 154, 155, 156, 158, 159, 160, 161, 162, 163, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 195, 196, 197, 198, 200, 201, 202, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 295, 296, 297, 298, 299, 301, 302, 303, 304, 305, 306, 307, 308, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 401, 402, 403, 405, 406, 407, 408, 409, 411, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 492, 493, 494, 495, 496, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 555, 556, 557, 558, 559, 560, 562, 563, 564, 565, 567, 568, 569, 570, 571, 574, 575, 576, 577, 579, 580, 581, 583, 584, 585, 586, 587, 588, 589, 590, 591, 592, 593, 595, 596, 598, 599, 600, 601, 602, 603, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 630, 631, 632, 633, 634, 636, 638, 639, 640, 641, 642, 643, 644, 645, 646, 647, 648, 649, 650, 652, 653, 654, 655, 656, 657, 658, 659, 660, 661, 662, 663, 664, 665, 667, 668, 669, 671, 672, 673, 674, 675, 677, 678, 679, 680, 681, 682, 683, 685, 686, 687, 688, 689, 690, 691, 692, 693, 694, 695, 696, 697, 698, 699, 700, 701, 702, 703, 704, 705, 706, 707, 708, 709, 710, 711, 712, 713, 714, 715, 716, 717, 718, 719, 720, 721, 722, 723, 724, 725, 726, 727, 728, 729, 730, 731, 732, 733, 734, 735, 736, 737, 738, 739, 740, 741, 742, 743, 744, 745, 746, 747, 748, 749, 752, 753, 755, 756, 757, 758, 759, 760, 761, 762, 763, 765, 766, 767, 768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 778, 779, 780, 781, 782, 783, 784, 785, 786, 787, 788, 789, 790, 795, 796, 797, 798, 799, 951, 952, 953, 954, 955, 956, 957, 958, 960, 962, 963, 964, 967, 968, 969, 970, 971, 973, 974, 975, 976, 977, 978, 980, 981, 982, 983, 985, 986, 987, 988, 989, 990, 991, 992, 993, 994, 995, 996, 997, 998, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023, 1024, 1025, 1026, 1027, 1028, 1030, 1031, 1032, 1033, 1034, 1035, 1036, 1037, 1038, 1040, 1041, 1042, 1043, 1044, 1045, 1046, 1047, 1048, 1049, 1050, 1051, 1052, 1053, 1054, 1055, 1056, 1057, 1058, 1059, 1060, 1062, 1063, 1064, 1065, 1066, 1067, 1068, 1069, 1070, 1071, 1072, 1073, 1075, 1076, 1077, 1078, 1079, 1081, 1082, 1083, 1084, 1085, 1086, 1087, 1089, 1091, 1092, 1093, 1094, 1095, 1096, 1097, 1098, 1102, 1103, 1104, 1105, 1106, 1107, 1108, 1109, 1110, 1111, 1112, 1113, 1115, 1117, 1118, 1119, 1121, 1122, 1123, 1124, 1125, 1126, 1127, 1128, 1130, 1132, 1133, 1134, 1135, 1136, 1137, 1138, 1139, 1140, 1141, 1142, 1143, 1144, 1145, 1146, 1147, 1149, 1151, 1152, 1153, 1155, 1156, 1157, 1158, 1159, 1160, 1161, 1162, 1163, 1164, 1165, 1166, 1167, 1168, 1169, 1171, 1172, 1173, 1174, 1175, 1176, 1177, 1178, 1179, 1180, 1181, 1182, 1183, 1184, 1185, 1186, 1187, 1188, 1189, 1190, 1191, 1192, 1193, 1194, 1195, 1196, 1197, 1198, 1199, 1200, 1201, 1202, 1203, 1204, 1205, 1207, 1209, 1211, 1212, 1215, 1216, 1217, 1219, 1221, 1222, 1223, 1224, 1225, 1226, 1227, 1228, 1229, 1230, 1231, 1232, 1233, 1234, 1235, 1236, 1237, 1238, 1240, 1241, 1242, 1243, 1245, 1246, 1247, 1248, 1249, 1250, 1253, 1255, 1256, 1257, 1261, 1262, 1264, 1265, 1266, 1267, 1268, 1269, 1270, 1271, 1272, 1275, 1276, 1277, 1279, 1280, 1281, 1282, 1283, 1284, 1285, 1286, 1287, 1288, 1290, 1291, 1292, 1293, 1294, 1295, 1300, 1301, 1302, 1303, 1304, 1305, 1306, 1307, 1308, 1309, 1310, 1312, 1313, 1314, 1315, 1316, 1317, 1318, 1319, 1320, 1322, 1324, 1325, 1326, 1327, 1328, 1329, 1330, 1331, 1332, 1334, 1340, 1342, 1343, 1347, 1349, 1350, 1352, 1353, 1354, 1355, 1356, 1357, 1358, 1359, 1360, 1361, 1362, 1363, 1365, 1366, 1367, 1370, 1371, 1372, 1373, 1374, 1376, 1377, 1378, 1379, 1381, 1382, 1383, 1385, 1386, 1388, 1389, 1390, 1391, 1393, 1394, 1395, 1396, 1397, 1399, 1400, 1401, 1403, 1404, 1405, 1407, 1408, 1409, 1410, 1411, 1413, 1414, 1415, 1416, 1418, 1419, 1420, 1421, 1422, 1424, 1425, 1426, 1428, 1429, 1430, 1431, 1432, 1434, 1435, 1436, 1437, 1438, 1439, 1440, 1441, 1442, 1443, 1445, 1447, 1448, 1452, 1453, 1454, 1455, 1456, 1457, 1458, 1461, 1462, 1463, 1465, 1466, 1467, 1468, 1469, 1470, 1473, 1474, 1476, 1480, 1481, 1482, 1483, 1484, 1487, 1492, 1493, 1494, 1497, 1498, 1499, 1500, 1501, 1502, 1503, 1504, 1506, 1507, 1508, 1510, 1515, 1516, 1517, 1518, 1519, 1520, 1521, 1522, 1525, 1529, 1530]

In [3]:
len(store_list)

1151

### Do the thing for one store

In [4]:
store_id = 7

# Read in csv file
traffic =pd.read_csv('./StoreHourlyTrafficClean/HourlyTrafficClean_' + str(store_id) + '.csv')

# Choose the used columns
traffic = traffic[['STR_ID', 'Date', 'SALEHOUR', 'IN_Clean', 'OUT_Clean']]

In [5]:
# Get date ranges

prediction_period = pd.date_range(start='2018-08-12', end='2018-10-31')
hours = [i for i in range(24)]

In [6]:
# Get DataFrame in the format for prediction

d_predict = {}
d_predict['trn_sls_dte'] = []
d_predict['SALEHOUR'] = []
d_predict['IN_Clean'] = []
d_predict['OUT_Clean'] = []
d_predict['STR_ID'] = []
for d in prediction_period:
    for h in hours:
        d_predict['trn_sls_dte'].append(d) 
        d_predict['SALEHOUR'].append(h)
        d_predict['IN_Clean'].append(0)
        d_predict['OUT_Clean'].append(0)
        d_predict['STR_ID'].append(store_id)

# Get DataFrame
df_predict = pd.DataFrame(d_predict)

# Change datetime to string in YYYY-MM-DD format
df_predict['Date'] = df_predict['trn_sls_dte'].dt.strftime('%Y-%m-%d')

# Select columns used.
df_predict = df_predict[['STR_ID', 'Date', 'SALEHOUR', 'IN_Clean', 'OUT_Clean']]

In [7]:
# Union to get all for each of the fscl_mn_id

traffic = pd.concat([traffic, df_predict])

In [8]:
if not os.path.exists('./StoreHourlyTrafficCleanPrediction'):
    os.mkdir('./StoreHourlyTrafficCleanPrediction')

traffic.to_csv('./StoreHourlyTrafficCleanPrediction/HourTrafficCleanPredict_' + str(store_id) + '.csv', index=False)

### For all stores

In [9]:
for store_id in store_list:

    # Read in csv file
    traffic =pd.read_csv('./StoreHourlyTrafficClean/HourlyTrafficClean_' + str(store_id) + '.csv')

    # Choose the used columns
    traffic = traffic[['STR_ID', 'Date', 'SALEHOUR', 'IN_Clean', 'OUT_Clean']]
    
    # Get date ranges
    prediction_period = pd.date_range(start='2018-08-12', end='2018-10-31')
    hours = [i for i in range(24)]

    # Get DataFrame in the format for prediction
    d_predict = {}
    d_predict['trn_sls_dte'] = []
    d_predict['SALEHOUR'] = []
    d_predict['IN_Clean'] = []
    d_predict['OUT_Clean'] = []
    d_predict['STR_ID'] = []
    for d in prediction_period:
        for h in hours:
            d_predict['trn_sls_dte'].append(d) 
            d_predict['SALEHOUR'].append(h)
            d_predict['IN_Clean'].append(0)
            d_predict['OUT_Clean'].append(0)
            d_predict['STR_ID'].append(store_id)

    # Get DataFrame
    df_predict = pd.DataFrame(d_predict)

    # Change datetime to string in YYYY-MM-DD format
    df_predict['Date'] = df_predict['trn_sls_dte'].dt.strftime('%Y-%m-%d')

    # Select columns used.
    df_predict = df_predict[['STR_ID', 'Date', 'SALEHOUR', 'IN_Clean', 'OUT_Clean']]

    # Union to get all for each of the fscl_mn_id
    traffic = pd.concat([traffic, df_predict])

    # Write to csv
    traffic.to_csv('./StoreHourlyTrafficCleanPrediction/HourTrafficCleanPredict_' + str(store_id) + '.csv', index=False)